<a href="https://colab.research.google.com/github/imr-framework/virtual-scanner/blob/master/virtualscanner/notebooks/Virtual_Scanner_2_0_MRpub_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Virtual Scanner 2.0 Sequence Simulation Portal

Gehua Tong, April 2022

*Image Acquisition and Reconstruction Group, Columbia MR Research Center, Columbia University*


---



Simulate your Pulseq sequence here! 

## Sequence selection
1. Demo : Single-shot EPI sequence 
2. User-uploaded (custom): any Pulseq .seq file can be uploaded. However, we recommend resolution equal to or less than that of the phantom.

## Phantom selection
1. Default cylindrical phantom with 3 planes of contrast spheres (PD, T1, T2 planes from the top down) and customizable parameter values. See phantom image [here](https://github.com/imr-framework/virtual-scanner/blob/master/virtualscanner/coms/coms_ui/static/phantom_pics/Numerical.png). 

2. User-uploaded (custom) phantoms in .mat format. 

## Hardware map selection 
1. Ideal hardware (zero off-resonance, uniform B1 maps with zero phase)
2. Real hardware maps (ACR 3T B0 map and 20 channel head coil B1 maps)
3. User-uploaded (custom) maps 


# Install and import simulator and other functions

In [ ]:
#  Installing, importing, ... 
!pip install git+https://github.com/imr-framework/virtual-scanner.git@2c9b320165f39d07e99e7d11d0fce5ba190d14dd

  Cloning https://github.com/imr-framework/virtual-scanner.git (to revision 2c9b320165f39d07e99e7d11d0fce5ba190d14dd) to /tmp/pip-req-build-2lqktly0
  Running command git clone -q https://github.com/imr-framework/virtual-scanner.git /tmp/pip-req-build-2lqktly0
  Running command git rev-parse -q --verify 'sha^2c9b320165f39d07e99e7d11d0fce5ba190d14dd'
  Running command git fetch -q https://github.com/imr-framework/virtual-scanner.git 2c9b320165f39d07e99e7d11d0fce5ba190d14dd
  Running command git checkout -q 2c9b320165f39d07e99e7d11d0fce5ba190d14dd


In [ ]:
# Install pypulseq ver. with custom "export_waveforms" function
!pip install pypulseq==1.3.1post1

In [ ]:
!pip install plotly==5.7.0

In [ ]:
!pip install pillow==9.1.0

# Import functions and define helper functions

In [ ]:
# Import all functions here
import matplotlib.pyplot as plt
import virtualscanner.server.simulation.bloch.phantom as pht
from pypulseq.Sequence.sequence import Sequence
import time
import multiprocessing as mp
import virtualscanner.server.simulation.bloch.pulseq_blochsim_methods as blcsim
from virtualscanner.server.simulation.bloch.pulseq_library import make_pulseq_epi_oblique
from google.colab import files
import numpy as np 
from scipy.io import savemat, loadmat
from ipywidgets import *
from IPython.display import clear_output, display
from google.colab import files
import math
import numpy as np
from matplotlib import pyplot as plt
from pypulseq.calc_rf_center import calc_rf_center
from scipy.io import savemat
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image


In [ ]:
def export_waveforms(seq, time_range=(0, np.inf)):
    """
    Plot `Sequence`.
    Parameters
    ----------
    time_range : iterable, default=(0, np.inf)
        Time range (x-axis limits) for all waveforms. Default is 0 to infinity (entire sequence).
    Returns
    -------
    all_waveforms: dict
        Dictionary containing all sequence waveforms and time array(s)
        The keys are listed here:
        't_adc' - ADC timing array [seconds]
        't_rf' - RF timing array [seconds]
        ''
        'adc' - ADC complex signal (amplitude=1, phase=adc phase) [a.u.]
        'rf' - RF complex signal []
        'gx' - x gradient []
        'gy' - y gradient []
        'gz' - z gradient []
    """
    # Check time range validity
    if not all([isinstance(x, (int, float)) for x in time_range]) or len(time_range) != 2:
        raise ValueError('Invalid time range')

    t0 = 0
    adc_t_all = np.array([])
    adc_signal_all = np.array([],dtype=complex)
    rf_t_all =np.array([])
    rf_signal_all = np.array([],dtype=complex)
    rf_t_centers = np.array([])
    rf_signal_centers = np.array([],dtype=complex)
    gx_t_all = np.array([])
    gy_t_all = np.array([])
    gz_t_all = np.array([])
    gx_all = np.array([])
    gy_all = np.array([])
    gz_all = np.array([])


    for block_counter in range(len(seq.dict_block_events)): # For each block
        block = seq.get_block(block_counter + 1)  # Retrieve it
        is_valid = time_range[0] <= t0 <= time_range[1] # Check if "current time" is within requested range.
        if is_valid:
            # Case 1: ADC
            if hasattr(block, 'adc'):
                adc = block.adc # Get adc info
                # From Pulseq: According to the information from Klaus Scheffler and indirectly from Siemens this
                # is the present convention - the samples are shifted by 0.5 dwell # OK

                t = adc.delay + (np.arange(int(adc.num_samples)) + 0.5) * adc.dwell
                adc_t = t0 + t
                adc_signal = np.exp(1j * adc.phase_offset) * np.exp(1j * 2 * np.pi * t * adc.freq_offset)
                adc_t_all = np.append(adc_t_all, adc_t)
                adc_signal_all = np.append(adc_signal_all, adc_signal)

            if hasattr(block, 'rf'):
                rf = block.rf
                tc, ic = calc_rf_center(rf)
                t = rf.t + rf.delay
                tc = tc + rf.delay
                #
                # sp12.plot(t_factor * (t0 + t), np.abs(rf.signal))
                # sp13.plot(t_factor * (t0 + t), np.angle(rf.signal * np.exp(1j * rf.phase_offset)
                #                                         * np.exp(1j * 2 * math.pi * rf.t * rf.freq_offset)),
                #           t_factor * (t0 + tc), np.angle(rf.signal[ic] * np.exp(1j * rf.phase_offset)
                #                                          * np.exp(1j * 2 * math.pi * rf.t[ic] * rf.freq_offset)),
                #           'xb')

                rf_t = t0 + t
                rf = rf.signal * np.exp(1j * rf.phase_offset) \
                                                        * np.exp(1j * 2 * math.pi * rf.t * rf.freq_offset)
                rf_t_all = np.append(rf_t_all, rf_t)
                rf_signal_all = np.append(rf_signal_all, rf)
                rf_t_centers = np.append(rf_t_centers, rf_t[ic])
                rf_signal_centers = np.append(rf_signal_centers, rf[ic])

            grad_channels = ['gx', 'gy', 'gz']
            for x in range(len(grad_channels)): # Check each gradient channel: x, y, and z
                if hasattr(block, grad_channels[x]): # If this channel is on in current block
                    grad = getattr(block, grad_channels[x])
                    if grad.type == 'grad':# Arbitrary gradient option
                        # In place unpacking of grad.t with the starred expression
                        g_t = t0 +  grad.delay + [0, *(grad.t + (grad.t[1] - grad.t[0]) / 2),
                                              grad.t[-1] + grad.t[1] - grad.t[0]]
                        g = 1e-3 * np.array((grad.first, *grad.waveform, grad.last))
                    else: # Trapezoid gradient option
                        g_t = t0 + np.cumsum([0, grad.delay, grad.rise_time, grad.flat_time, grad.fall_time])
                        g = 1e-3 * grad.amplitude * np.array([0, 0, 1, 1, 0])

                    if grad.channel == 'x':
                        gx_t_all = np.append(gx_t_all, g_t)
                        gx_all = np.append(gx_all,g)
                    elif grad.channel == 'y':
                        gy_t_all = np.append(gy_t_all, g_t)
                        gy_all = np.append(gy_all,g)
                    elif grad.channel == 'z':
                        gz_t_all = np.append(gz_t_all, g_t)
                        gz_all = np.append(gz_all,g)


        t0 += seq.arr_block_durations[block_counter] # "Current time" gets updated to end of block just examined

    all_waveforms = {'t_adc': adc_t_all, 't_rf': rf_t_all, 't_rf_centers': rf_t_centers,
                     't_gx': gx_t_all, 't_gy':gy_t_all, 't_gz':gz_t_all,
                     'adc': adc_signal_all, 'rf': rf_signal_all, 'rf_centers': rf_signal_centers,'gx':gx_all, 'gy':gy_all, 'gz':gz_all,
                     'grad_unit': '[kHz/m]', 'rf_unit': '[Hz]', 'time_unit':'[seconds]'}

    return all_waveforms

In [ ]:
def display_seq_interactive(all_waveforms,time_range=(0,np.inf)): 
  fig = make_subplots(rows=3, cols=2,
      subplot_titles=("RF magnitude", "Gx", "RF phase", "Gy", "ADC", "Gz"),shared_xaxes='all',row_heights=[10,10,10])

  fig.add_trace(go.Scatter(x=all_waveforms['t_rf'],y=np.absolute(all_waveforms['rf']),mode='lines',name='RF magnitude',line=dict(color='blue',width=2)),
                row=1,col=1)
  fig.add_trace(go.Scatter(x=all_waveforms['t_rf'],y=np.angle(all_waveforms['rf']),mode='lines',name='RF phase',line=dict(color='gray',width=2)),
                row=2,col=1)
  fig.add_trace(go.Scatter(x=all_waveforms['t_adc'],y=np.angle(all_waveforms['adc']),mode='markers',name='ADC with phase',line=dict(color='red',width=2)),
                row=3,col=1)
  fig.add_trace(go.Scatter(x=all_waveforms['t_gx'],y=all_waveforms['gx'],mode='lines',name='Gx',line=dict(color='green', width=2)), 
                row=1,col=2)
  fig.add_trace(go.Scatter(x=all_waveforms['t_gy'],y=all_waveforms['gy'],mode='lines',name='Gy',line=dict(color='orange', width=2)), 
                row=2,col=2)
  fig.add_trace(go.Scatter(x=all_waveforms['t_gz'],y=all_waveforms['gz'],mode='lines',name='Gz',line=dict(color='purple', width=2)), 
                row=3,col=2)
  
  fig.update_xaxes(title_text="Time (seconds)", row=3, col=1,range=time_range)
  fig.update_xaxes(title_text="Time (seconds)", row=3, col=2,range=time_range)
  fig.update_yaxes(title_text=all_waveforms['rf_unit'],row=1,col=1)
  fig.update_yaxes(title_text='[rads]',row=2,col=1)
  fig.update_yaxes(title_text='[rads]',row=3,col=1)
  
  
  fig.show()

In [ ]:
def display_vs_phantom(pht, zindex=0):
  p = pht
  fig = make_subplots(rows=1, cols=3,
        subplot_titles=("T1 (seconds)", "T2 (seconds)", "PD (a.u.)"))
  fig.add_trace(go.Heatmap(z=p.T1map[:,:,zindex],colorbar=dict(len=1.05,x=0.3)),row=1,col=1)
  fig.add_trace(go.Heatmap(z=p.T2map[:,:,zindex],colorbar=dict(len=1.05,x=0.65)),row=1,col=2)
  fig.add_trace(go.Heatmap(z=p.PDmap[:,:,zindex],colorbar=dict(len=1.05,x=1)),row=1,col=3)

  fig.update_xaxes(showticklabels=False)
  fig.update_yaxes(showticklabels=False)
  fig.show()

In [ ]:
def display_b0_b1_maps(scanner_info,zindex=0):
  b0 = scanner_info['B0']
  tx = scanner_info['B1tx']
  rx = scanner_info['B1rx']

  fig = make_subplots(rows=2,cols=3,
                      subplot_titles=("B0 (Hz)", "B1 Tx magnitude", "B1 Rx magnitude","","B1 Tx phase (rad)","B1 Rx phase (rad)"))
  
  fig.update_layout(height=1000,width=1200)

  fig.add_trace(go.Heatmap(z=b0[:,:,zindex],colorbar=dict(len=0.4,x=0.29,y=0.81)),row=1,col=1)
  fig.add_trace(go.Heatmap(z=np.absolute(tx[:,:,zindex]),colorbar=dict(len=0.4,x=0.64,y=0.81)),row=1,col=2)
  fig.add_trace(go.Heatmap(z=np.absolute(rx[:,:,zindex]),colorbar=dict(len=0.4,x=1,y=0.82)),row=1,col=3)
  fig.add_trace(go.Heatmap(z=np.angle(tx[:,:,zindex]),colorbar=dict(len=0.4,x=0.64,y=0.19)),row=2,col=2)
  fig.add_trace(go.Heatmap(z=np.angle(rx[:,:,zindex]),colorbar=dict(len=0.4,x=1,y=0.19)),row=2,col=3)

  for r in range(2):
    for c in range(3):
      fig.update_yaxes(showgrid=False,showticklabels=False,row=r+1,col=c+1)
      fig.update_xaxes(showgrid=False,showticklabels=False,row=r+1,col=c+1)



  fig.show()

In [ ]:
# Grab files
!wget https://raw.githubusercontent.com/imr-framework/virtual-scanner/dev/virtualscanner/notebooks/data/epi_pypulseq_32.seq
!wget https://raw.githubusercontent.com/imr-framework/virtual-scanner/dev/virtualscanner/notebooks/data/b0_map_smaller_Hz.mat
!wget https://raw.githubusercontent.com/imr-framework/virtual-scanner/dev/virtualscanner/notebooks/data/rx_maps_ds_16_32.mat

--2022-05-02 02:11:12--  https://raw.githubusercontent.com/imr-framework/virtual-scanner/dev/virtualscanner/notebooks/data/epi_pypulseq_32.seq
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32889 (32K) [text/plain]
Saving to: ‘epi_pypulseq_32.seq.2’

epi_pypulseq_32.seq 100%[===================>]  32.12K  --.-KB/s    in 0.001s  

2022-05-02 02:11:12 (41.8 MB/s) - ‘epi_pypulseq_32.seq.2’ saved [32889/32889]

--2022-05-02 02:11:12--  https://raw.githubusercontent.com/imr-framework/virtual-scanner/dev/virtualscanner/notebooks/data/b0_map_smaller_Hz.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443.

# Make or load a Pulseq file and visualize the sequence
**Step 1: Select the sequence**

Using the form on the right:
* Select "EPI" to simulate a default single-shot EPI sequence 
* Select "Custom" to simulate your own file

After selecting, click run. Upload your custom .seq file if prompted.

In [ ]:
#@title Sequence Option
seq_type  = 'EPI' #@param["EPI","Custom"] 
# Run this cell to continue setting up the sequence 
if seq_type == 'EPI':
  print('Setting up EPI sequence...')
  #@title EPI Parameters
  #@markdown You only need to fill the following fields for the demo EPI simulation.
  #@markdown Timings are in units of (ms) and spatial units are in (mm).
  N =  16#@param {type: "integer"} 
  FOV = 250 #@param {type: "number"}
  TR = 1000 #@param {type: "number"}
  TE = 50 #@param {type: "number"}

  seq, ro_dirs, ro_order = make_pulseq_epi_oblique(FOV*1e-3,N,thk=5e-3,fa=90,tr=TR*1e-3,te=TE*1e-3,slice_locs=[0])
  print("EPI sequence generated.")
if seq_type == 'Custom':
  uploaded = files.upload()
  seqname = list(uploaded.keys())[0]
  seq = Sequence()
  seq.read(seqname)
#@markdown If choosing "Custom", click "Choose Files" below to upload your .seq file.

Setting up EPI sequence...
ug_fe:  (1, 0, 0)
ug_pe:  (0, 1, 0)
ug_ss:  (0, 0, 1)
ADC delay:  8e-05
EPI sequence (oblique) constructed
EPI sequence generated.


**Step 2**: Display the sequence

In [ ]:
# Display sequence
#@title Display sequence 
#@markdown Choose the desired time range in [seconds]. Then run this cell.


#all_waveforms = export_waveforms(seq, time_range=[start_time,end_time])
min_time = 0.04 #@param {type:"number"}
max_time = 0.1#@param {type:"number"}
all_waveforms = export_waveforms(seq,time_range=[min_time,max_time])
display_seq_interactive(all_waveforms,time_range=[min_time,max_time])

# Make a phantom and inspect it


In [ ]:
#@title Phantom Options

#@markdown Choose your phantom type and then skip to the corresponding section
#@markdown for the next steps.

phantom_type = "Default" #@param ["Default","Custom"]

#@markdown # Default Phantom 
#@markdown One of the three planes of the default cylindrical phantom may be selected.
#@markdown Choose your own parameters and
#@markdown then run the cell to display the phantom maps. 

if phantom_type == "Default":
  zi = 0
  #@markdown The phantom background has PD = 0 and the main cylindrical fill has 
  #@markdown T1 = 500 ms, T2 = 100 ms, and PD = 1. The same default values are also used for
  #@markdown the contrast sphere for the non-varying parameters. For example, 
  #@markdown T1 = 500 ms and T2 = 100 ms are used for the PD plane where only PD varies
  #@markdown among the spheres.

  plane_option = "T2" #@param ["T1","T2","PD"]
  phantom_matrix_size =  16#@param {type:"integer"}
  # Phantom size
  Npht = phantom_matrix_size 
  FOV = 0.25
  #@markdown **T1 plane** (4 spheres)
  T1a = 1.5 #@param {type:"number"}
  T1b = 0.6 #@param {type:"number"}
  T1c = 0.25 #@param {type:"number"}
  T1d = 0.1 #@param {type:"number"}

  #@markdown **T2 plane** (4 spheres)
  T2a =  1#@param {type:"number"}
  T2b = 0.5 #@param {type:"number"}
  T2c = 0.08 #@param {type:"number"}
  T2d = 0.01 #@param {type:"number"}

  #@markdown **PD plane** (3 spheres)
  PDa =  1#@param {type:"number"}
  PDb = 0.75 #@param {type:"number"}
  PDc = 0.5 #@param {type:"number"}

  # Phantom parameters 
  T1T2PD0 = [0.5,0.05,1] # Default values 
  T1T2PD1 = [0.5,0.05,1] # Main cylinder fill  

  # The following have been customized 
  PDs = [PDa,PDb,PDc]
  T1s = [T1a,T1b,T1c,T1d]
  T2s = [T2a,T2b,T2c,T2d]

  if plane_option == 'T1':
    p = pht.makeCustomCylindricalPhantom(T1T2PD0, PDs, T1s, T2s, T1T2PD1=T1T2PD1, dim=2, n=Npht, dir='z', loc=0, fov=FOV)
  elif plane_option == 'T2':
    p = pht.makeCustomCylindricalPhantom(T1T2PD0, PDs, T1s, T2s, T1T2PD1=T1T2PD1, dim=2, n=Npht, dir='z', loc=-FOV/3, fov=FOV)
    p.loc = (0,0,0)
    p.Zs = [0]
  elif plane_option == 'PD':
    p = pht.makeCustomCylindricalPhantom(T1T2PD0, PDs, T1s, T2s, T1T2PD1=T1T2PD1, dim=2, n=Npht, dir='z', loc=FOV/3, fov=FOV)
    p.loc = (0,0,0)
    p.Zs = [0]
  
#@markdown After entering the desired parameters, run this cell. 
#@markdown The phantom will be constructed and displayed. 

#@markdown # Custom Phantom

#@markdown First, choose voxel size in millimeters:
vsize = 1 #@param {type:"number"}

#@markdown Then, run this cell. When prompted, upload a .mat file with the following matrices:
#@markdown * T1map (seconds)
#@markdown * T2map (seconds)
#@markdown * PDmap (arbitrary unit)

#@markdown All three matrices should be the same size and have three dimensions (x,y,z). 
#@markdown The phantom will then be constructed. If you are using a 3D phantom, 
#@markdown please choose the z (3rd) index for the 2D display, happening next.  

if phantom_type == "Custom":
    z_index = 1 #@param 
    zi = z_index - 1 
    uploaded = files.upload()
    matpath = list(uploaded.keys())[0]
    t1map = loadmat(matpath)['T1map']
    t2map = loadmat(matpath)['T2map']
    pdmap = loadmat(matpath)['PDmap']

    p = pht.Phantom(T1map=t1map,T2map=t2map,PDmap=pdmap,vsize=vsize,dBmap=0)

#@markdown The phantom will be displayed after loading.

<class 'numpy.ndarray'>


# Display phantom maps
Run the cell to display the T1, T2 and PD maps of the phantom for closer inspection. Note that the default phantom is always a 2D plane  

In [ ]:
display_vs_phantom(p,zi)

# Set up the simulation 


In [ ]:
dfmap = map_scale * loadmat('b0_map_smaller_Hz.mat')['b0_32_Hz']
print(np.max(dfmap))
print(np.min(dfmap))

93.26207870568156
-35.91876521624458


In [ ]:
# Hardware setup
# Prompt the user to upload hardware map (show specifications clearly)
# Or: use a preset one 
# Or: use ideal 

phantom_shape = np.shape(p.T1map)

#@markdown # Hardware map options
#@markdown Choose the system B0, B1+, and B1- maps. "Ideal" means the maps are uniform across 
#@markdown the FOV. Then run the cell to visualize these maps. 

#@markdown ## B0
#@markdown * The "Ideal" option assume no off-resonance at all points of the phantom (df=0)
#@markdown * THe "ACR 3T" option can only be used with 
#@markdown phantoms of size [16,16,1] or [32,32,1]. 
#@markdown * For the "Custom" option, upload a .mat file with a varaible named 
#@markdown "dfmap" with the same size as the phantom ([N_pht,N_pht,1] for the 
#@markdown default phantom) in units of [Hz].

#@markdown The parameter "B0_scale" can be used to scale the bandwidth of the "ACR 3T" or 
#@markdown "Custom" B0 map. With `B0_scale` = 1, the bandwidth is from -36 Hz to +93 Hz. 
B0_type = "Ideal" #@param ["Ideal","ACR 3T","Custom"]
B0_scale = 1 #@param {type:"number"}

if B0_type == "Ideal":
  dfmap = np.zeros(phantom_shape)
elif B0_type == "ACR 3T": 
  dfmap = map_scale * loadmat('b0_map_smaller_Hz.mat')['b0_32_Hz']
  dfmap = np.reshape(dfmap, (dfmap.shape[0],dfmap.shape[1],1))
  if p.PDmap.shape[2] > 1:
    raise ValueError("This B0 map is 2D; use a phantom with shape [M,M,1]")
  if p.PDmap.shape == (16,16,1):
    dfmap = dfmap[0::2,0::2,0]
    dfmap = np.reshape(dfmap, (dfmap.shape[0],dfmap.shape[1],1))

  elif p.PDmap.shape != (32,32,1):
    raise ValueError("When using the ACR 3T map, the phantom size must be 16 x 16 x 1 or 32 x 32 x 1")
elif B0_type == "Custom":
  print("Upload B0 map file:")
  uploaded = files.upload()
  b0_path = list(uploaded.keys())[0]
  dfmap = map_scale * loadmat(b0_path)['dfmap']
  
  if len(dfmap.shape) == 2: 
     dfmap = np.reshape(dfmap, (dfmap.shape[0],dfmap.shape[1],1))
  if dfmap.shape != p.PDmap.shape:
    raise ValueError("Uploaded B0 map shape must be the same as the phantom shape")

#@markdown ## B1+ (transmit coil)
#@markdown * The "Ideal" option assume no off-resonance at all points of the phantom (df=0)
#@markdown * THe "20 ch head coil" option can only be used with 
#@markdown phantoms of size [16,16,1] or [32,32,1]. 
#@markdown * For the "Custom" option, upload a .mat file with a varaible named 
#@markdown "txmap" with the same size as the phantom ([N_pht,N_pht,1] for the 
#@markdown default phantom). A numerical value of 1 indicates ideal transmission. 
#@markdown This map can be complex.  

#@markdown The parameter "Tx_scale" can be used to scale "20 ch head coil" Tx map. 

tx_type = "Ideal" #@param ["Ideal","20 ch head coil","Custom"]
tx_channel_for_head_coil = 7 #@param {type:"slider", min:1, max:20, step:1}
Tx_scale = 2 #@param {type:"number"}

c = tx_channel_for_head_coil + 1 
if tx_type == "Ideal": 
  txmap = np.ones(phantom_shape) 
elif tx_type == "20 ch head coil":
  txmap = loadmat('rx_maps_ds_16_32.mat')['rx_maps_32_norm'][:,:,c]
  txmap = Tx_scale * np.reshape(txmap, (txmap.shape[0],txmap.shape[1],1))
  if p.PDmap.shape[2] > 1:
    raise ValueError("This B0 map is 2D; use a phantom with shape [M,M,1]")
  if p.PDmap.shape == (16,16,1):
    txmap = txmap[0::2,0::2,0]
    txmap = np.reshape(txmap, (txmap.shape[0],txmap.shape[1],1))

  elif p.PDmap.shape != (32,32,1):
    raise ValueError("When using the 20 ch head coil map, the phantom size must be 16 x 16 x 1 or 32 x 32 x 1")

elif tx_type == "Custom":
  print('Upload transmit B1 coil file:')
  uploaded = files.upload()
  matpath = list(uploaded.keys())[0]
  txmap = loadmat(matpath)['txmap']

  if len(txmap.shape) == 2: 
     txmap = np.reshape(txmap, (txmap.shape[0],txmap.shape[1],1))
  if txmap.shape != p.PDmap.shape:
    raise ValueError("Uploaded B1 tx map shape must be the same as the phantom shape")

#@markdown ## B1- (receive coil)

#@markdown For the "Custom" option, upload a .mat file with a varaible named 
#@markdown "rxmap" with the same size as the phantom ([N_pht,N_pht,1] for the 
#@markdown default phantom). A numerical value of 1 indicates ideal received signal. 
#@markdown This map can be complex.  

#@markdown The parameter "Tx_scale" can be used to scale "20 ch head coil" Tx map. 

rx_type = "20 ch head coil" #@param ["Ideal","20 ch head coil","Custom"]
rx_channel_for_head_coil = 13 #@param {type:"slider", min:1, max:20, step:1}
Rx_scale = 2 #@param {type:"number"}

d = rx_channel_for_head_coil + 1 
if rx_type == "Ideal": 
  rxmap = np.ones(phantom_shape) 
elif rx_type == "20 ch head coil":
  rxmap = loadmat('rx_maps_ds_16_32.mat')['rx_maps_32_norm'][:,:,d]
  rxmap = Rx_scale * np.reshape(rxmap,(rxmap.shape[0],rxmap.shape[1],1))
  if p.PDmap.shape[2] > 1:
    raise ValueError("This B0 map is 2D; use a phantom with shape [M,M,1]")
  if p.PDmap.shape == (16,16,1):
    rxmap = rxmap[0::2,0::2,0]
    rxmap = np.reshape(rxmap,(rxmap.shape[0],rxmap.shape[1],1))
  elif p.PDmap.shape != (32,32,1):
    raise ValueError("When using the 20 ch head coil map, the phantom size must be 16 x 16 x 1 or 32 x 32 x 1")

elif rx_type == "Custom":
  print('Upload receive B1 coil file')
  uploaded = files.upload()
  matpath = list(uploaded.keys())[0]
  rxmap = loadmat(matpath)['rxmap']
  if len(rxmap.shape) == 2: 
    rxmap = np.reshape(rxmap,(rxmap.shape[0],rxmap.shape[1],1))
  if rxmap.shape != p.PDmap.shape:
    raise ValueError("Uploaded B1 rx map shape must be the same as the phantom shape")


# Set up hardware maps 
scanner_info = {'B0': dfmap, 'B1tx':txmap, 'B1rx':rxmap}

# Display hardware 
display_b0_b1_maps(scanner_info)



# Run the simulation

In [ ]:
# Run simulation using multiprocessing
myseq = seq
myphantom = p 

# Time the code: Tic
start_time = time.time()
# Store seq info
seq_info = blcsim.store_pulseq_commands(myseq)
# Get list of locations from phantom
loc_ind_list = myphantom.get_list_inds()
# Initiate multiprocessing pool
pool = mp.Pool(mp.cpu_count())
# Parallel simulation
sg_type = 'Default'


scanner_info = {'B0': dfmap, 'B1tx':txmap, 'B1rx':rxmap}

df = 0 
results = pool.starmap_async(blcsim.sim_single_spingroup_v2,
                              [(loc_ind, df, myphantom, seq_info, scanner_info, sg_type) for loc_ind in loc_ind_list]).get()
pool.close()
# Add up signal across all SpinGroups
my_signal = np.sum(results, axis=0)
savemat('simulated_signals.mat', {'signal':my_signal})

# Time the code: Toc
print("Time used: %s seconds" % (time.time() - start_time))

Time used: 23.743250846862793 seconds


# Reconstruct the images
We reconstruct images for the default EPI sequence. For custom sequences, skip this cell and run the next one to export your data as a .mat file. 

In [ ]:
  # Reconstruct EPI signals 
  #kk[:,0::2] = my_signal[::-1,0::2]
  im = np.fft.fftshift(np.fft.ifft2(np.transpose(my_signal)))
  fig = px.imshow(np.absolute(im),binary_string=True)
  fig.update_xaxes(showticklabels=False)
  fig.update_yaxes(showticklabels=False)

# Save and download simulated raw data


In [ ]:
savemat("simulated_data_for_download.mat", {"data":my_signal})
files.download('simulated_data_for_download.mat')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>